<a href="https://colab.research.google.com/github/ratmcu/wiki_ner/blob/master/info_box.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pyahocorasick
!pip install fuzzyset
from ahocorasick import Automaton
import fuzzyset
import urllib
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
import nltk
import re
import pandas as pd
import numpy as np
nltk.download('punkt')
!pip install wget
!pip install spacy
!pip install regex
!python -m spacy download en_core_web_sm
import spacy
import regex

import os
import wget
colabimporturl = 'https://github.com/ratmcu/colaboratory_import/raw/master/colabimport.py'
filename = colabimporturl.split("/")[-1].split("?")[0]
if os.path.isfile(filename):
    os.remove(filename)
wget.download(colabimporturl)

import colabimport
colabimport.get_notebook('https://github.com/ratmcu/wiki_ner/blob/master/reusable_annotator.ipynb?raw=true')
colabimport.get_notebook('https://github.com/ratmcu/wiki_ner/blob/master/info_box.ipynb?raw=true')
# import io, os, sys, types
from reusable_annotator import PageContents
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [0]:
class InfoCard():
    '''accepts a beautiful soup html table from a wikipedia page and scrapes the table to a dictionary'''
    def __init__(self, page_content):
        self.info_table = {}
        if not page_content.table:
            raise Exception('page content is incomplete')
        for table_entry in page_content.table_entry_list:
            try:
                left_col = table_entry.find('th', attrs={'scope': 'row'})
                right_col = left_col.next_sibling
                self.info_table[self._row_filter(left_col.text)] = self._get_text_parts(self._get_kids(right_col))
            except:
                pass
        self.info_table_unfiltered = self.info_table
        self.info_table = self._filter_info_scrapes(self.info_table)
        logging.debug('info card is scraped successfully')
        logging.debug(self.info_table)
    
    def _row_filter(self, text):
        return re.sub('\xa0', ' ', text)
        
        
    def _get_kids(self, html_mother):
        kid_list = []
        try:
            kids = html_mother.children
            for kid in kids:
                kid_list.append(self._get_kids(kid))
        except:
            return html_mother
        return kid_list 

    def _get_text_parts(self, text_lists):
        text_parts = []
        if type(text_lists) == list:
            for element in text_lists:
                text_parts.extend(self._get_text_parts(element))
        else:
            text_parts.append(text_lists)
        return(text_parts)

    def _filter_info_scrapes(self, scape_dict):
        mask_dict = {}
        for key, val in scape_dict.items():
            mask_dict[key] = []
            for i, element in enumerate(scape_dict[key]):
                name = ''
                lst = [' '+nm[0] for nm in [part.split('(') for part in element.split(' ')] if nm[0]!='']
                element = ''.join(lst).strip()
                fnd = [c in element for c in u'[]\n\xa0']
                if not True in fnd:
                    mask_dict[key].append(element)  
        return mask_dict    

In [0]:
#experiment_code
# pg = PageContents('https://en.wikipedia.org/wiki/Barack_Obama')
pg = PageContents('https://en.wikipedia.org/wiki/Donald_Trump')
# pg = PageContents('http://en.wikipedia.org/wiki/Gajendra_Singh_Rajukhedi')

info_card = InfoCard(pg)
print(info_card.info_table_unfiltered)
print(info_card.info_table)


info card is scraped successfully
{'Vice President': ['Mike Pence'], 'Preceded by': ['Barack Obama'], 'Born': ['Donald John Trump', ' (', '1946-06-14', ') ', 'June 14, 1946', ' (age\xa073)', 'Queens', ', ', 'New York City'], 'Political party': ['Republican', ' (1987–1999, 2009–2011, 2012–present)'], 'Other politicalaffiliations': ['\n', 'Democratic', ' (until 1987, 2001–2009)', '\n', 'Reform', ' (1999–2001)', '\n', 'Independent', ' (2011–2012)', '\n'], 'Spouse(s)': ['\n', 'Ivana Zelníčková', '(', 'm.', '\xa0', '1977', '; ', 'div.', '\xa0', '1992', ')', '\n', 'Marla Maples', '(', 'm.', '\xa0', '1993', '; ', 'div.', '\xa0', '1999', ')', '\n', 'Melania Knauss', ' (', 'm.', '\xa0', '2005', ')', '\n'], 'Children': ['\n', 'Donald Jr.', '\n', 'Ivanka', '\n', 'Eric', '\n', 'Tiffany', '\n', 'Barron', '\n'], 'Parents': ['Fred Trump', 'Mary Anne MacLeod'], 'Relatives': ['Trump family'], 'Residence': ['\n', 'White House', ' (official)', '\n', 'Trump Tower', ' (personal)', '\n', 'Full list', '\n'],

In [0]:
# can we use snorkle to get this entity scraping done??? 
# if so it'd be better to have the lable functions in a ready way to generally working on all the pages!!

class PrivateEntities():
    '''given a dictionary of a side bar it is processed for private entities of interest'''
    def __init__(self, info_card):
        self.info_dict = info_card.info_table
        # print(info_card.info_table)
        self._get_entity_dict()
        self._extract_entities()
        
    def _extract_entities(self):
        # TREAT everyone the same? or call a seperate function per each entity?
        # leave the not found entities as empty lists so we can use the html scraper to fill them (usually the name of the person)
        # lets treat everything generally, ultimate filtering happens at the comparison stage(if theres one)
        for entity_key in self.entity_dict.keys():
#             print(entity_key, self.entity_dict[entity_key])
            if self.entity_dict[entity_key] == []:
                continue
            for tiny_dict in self.entity_dict[entity_key]:
                self._pick_entity(tiny_dict, entity_key)
            logging.debug(self.entity_dict[entity_key])
    
    def _pick_entity(self, entity_list, entity_key):
        nlp = spacy.load("en_core_web_sm")
#         print(entity_list)
#         Find named entities, phrases and concepts
#         entity_list.append([])
#         print(entity_list[0][0])
        #TODO: add a check for the NAME too
        if entity_key == 'BIRTH_PLACE':
            mask_str = ''
            for scrape in self.info_dict[entity_list['dict_key']]:
                doc = nlp(str(scrape))
                if doc.ents and doc.ents[0].label_ == 'DATE':
                    mask_str = mask_str + 'd'
                elif str(scrape) == '' or str(scrape) == ',':
                    mask_str = mask_str + 's'
                elif self._is_name(str(scrape)):
                    mask_str = mask_str + 't'
                else:
#                     print(scrape)
                    mask_str = mask_str + 'u'
#             print(mask_str)
            indices = self._get_target_indices([r'[s][t]',r'[d][t]'], mask_str)  
            for index in indices:
                entity_list['entity_list'].append(self.info_dict[entity_list['dict_key']][index])
        else:    
            for scrape in self.info_dict[entity_list['dict_key']]:
                if entity_list['type'] == 'PERSON': #spacy skips certain parts of a name, so we use this trick instead
                    doc = nlp('was named '+str(scrape))
                    logging.debug(scrape)
                else:
                    doc = nlp(str(scrape))
                if doc.ents and doc.ents[0].label_ == entity_list['type']:   
                    entity_list['entity_list'].append(doc.ents[0].text)
                    logging.debug('appended entity: ', doc.ents[0].text, doc.ents[0].label_, len(doc.ents), doc.ents)
                elif not self._entity_noise(entity_key, str(scrape)):
                    entity_list['entity_list'].append(str(scrape))
                else:
                    pass
                
    def _get_target_indices(self, pattern_list, mask_str):
        '''TODO: if the target is inside the pattern'''
        indices = []
        for pattern in pattern_list:
            m_iter = re.finditer(pattern, mask_str)
            if m_iter:
                for m in m_iter: 
                    indices.append(m.start()+1)
        return indices        
        
    def _entity_noise(self, entity_key, text):
        try:
            return getattr(self, '_filter_{0}'.format(entity_key))(text)
        except:
            logging.error('filter function for {0} not found'.format(entity_key))
            return True
    
    def _get_entity_dict(self):
        self.entity_dict = {}
        self.entity_dict['NAME'] = []
        self.entity_dict['BIRTH_DATE'] = []
        self.entity_dict['BIRTH_PLACE'] = []
        self.entity_dict['CHILDREN'] = []
        self.entity_dict['SPOUSES'] = []
        self.entity_dict['PARENTS'] = []
        self.entity_dict['EDUCATION'] = []
        if 'Born' in self.info_dict.keys():
            self.entity_dict['NAME'] += [{'dict_key':'Born', 'type':'PERSON', 'entity_list':[]}]
            self.entity_dict['BIRTH_DATE'] += [{'dict_key':'Born', 'type':'DATE', 'entity_list':[]}]
            self.entity_dict['BIRTH_PLACE'] += [{'dict_key':'Born', 'type':'GPE', 'entity_list':[]}]
        if 'Born:' in self.info_dict.keys():
            self.entity_dict['NAME'] += [{'dict_key':'Born:', 'type':'PERSON', 'entity_list':[]}]
            self.entity_dict['BIRTH_DATE'] += [{'dict_key':'Born:', 'type':'DATE', 'entity_list':[]}]
            self.entity_dict['BIRTH_PLACE'] += [{'dict_key':'Born:', 'type':'GPE', 'entity_list':[]}]
        
        if 'Birth name' in self.info_dict.keys():
            self.entity_dict['NAME'] += [{'dict_key':'Birth name', 'type':'PERSON', 'entity_list':[]}]
        
        if 'Date of birth' in self.info_dict.keys():
            self.entity_dict['BIRTH_DATE'] += [{'dict_key':'Date of birth', 'type':'DATE', 'entity_list':[]}] 
        
        if 'Place of birth' in self.info_dict.keys():
            self.entity_dict['BIRTH_PLACE'] += [{'dict_key':'Place of birth', 'type':'GPE', 'entity_list':[]}] 
        
        if 'Children' in self.info_dict.keys():
            self.entity_dict['CHILDREN'] += [{'dict_key':'Children', 'type':'PERSON', 'entity_list':[]}]
            
        if 'Spouse(s)' in self.info_dict.keys():
            self.entity_dict['SPOUSES'] += [{'dict_key':'Spouse(s)', 'type':'PERSON', 'entity_list':[]}]
        if 'Spouses' in self.info_dict.keys():
            self.entity_dict['SPOUSES'] += [{'dict_key':'Spouses', 'type':'PERSON', 'entity_list':[]}]
        if 'Spouse' in self.info_dict.keys():
            self.entity_dict['SPOUSES'] += [{'dict_key':'Spouse', 'type':'PERSON', 'entity_list':[]}]
            
        if 'Parents' in self.info_dict.keys():
            self.entity_dict['PARENTS'] += [{'dict_key':'Parents', 'type':'PERSON', 'entity_list':[]}]
        if 'Parent' in self.info_dict.keys():
            self.entity_dict['PARENTS'] += [{'dict_key':'Parent', 'type':'PERSON', 'entity_list':[]}]
        if 'Parent(s)' in self.info_dict.keys():
            self.entity_dict['PARENTS'] += [{'dict_key':'Parent(s)', 'type':'PERSON', 'entity_list':[]}]
        if 'Father' in self.info_dict.keys():
            self.entity_dict['PARENTS'] += [{'dict_key':'Father', 'type':'PERSON', 'entity_list':[]}]
        if 'Mother' in self.info_dict.keys():
            self.entity_dict['PARENTS'] += [{'dict_key':'Mother', 'type':'PERSON', 'entity_list':[]}]
        if 'Father’s name' in self.info_dict.keys():
            self.entity_dict['PARENTS'] += [{'dict_key':'Father’s name', 'type':'PERSON', 'entity_list':[]}]
        if 'Mother’s name' in self.info_dict.keys():
            self.entity_dict['PARENTS'] += [{'dict_key':'Mother’s name', 'type':'PERSON', 'entity_list':[]}]
            
        if 'Education' in self.info_dict.keys():
            self.entity_dict['EDUCATION'] += [{'dict_key':'Education', 'type':'ORG', 'entity_list':[]}]
        if 'Alma mater' in self.info_dict.keys():
            self.entity_dict['EDUCATION'] += [{'dict_key':'Alma mater', 'type':'ORG', 'entity_list':[]}]
        if 'Alma\xa0mater' in self.info_dict.keys():
            self.entity_dict['EDUCATION'] += [{'dict_key':'Alma\xa0mater', 'type':'ORG', 'entity_list':[]}]
        if 'Almat mater' in self.info_dict.keys():
            self.entity_dict['EDUCATION'] += [{'dict_key':'Almat mater', 'type':'ORG', 'entity_list':[]}]
        if 'Law School' in self.info_dict.keys():
            self.entity_dict['EDUCATION'] += [{'dict_key':'Law School', 'type':'ORG', 'entity_list':[]}]
        if 'School' in self.info_dict.keys():
            self.entity_dict['EDUCATION'] += [{'dict_key':'School', 'type':'ORG', 'entity_list':[]}]
        if 'Schools' in self.info_dict.keys():
            self.entity_dict['EDUCATION'] += [{'dict_key':'Schools', 'type':'ORG', 'entity_list':[]}]
        if 'High school' in self.info_dict.keys():
            self.entity_dict['EDUCATION'] += [{'dict_key':'High school', 'type':'ORG', 'entity_list':[]}]
        if 'High school:' in self.info_dict.keys():
            self.entity_dict['EDUCATION'] += [{'dict_key':'High school:', 'type':'ORG', 'entity_list':[]}]
        if 'College' in self.info_dict.keys():
            self.entity_dict['EDUCATION'] += [{'dict_key':'College', 'type':'ORG', 'entity_list':[]}]
        if 'Colleges' in self.info_dict.keys():
            self.entity_dict['EDUCATION'] += [{'dict_key':'Colleges', 'type':'ORG', 'entity_list':[]}]
        if 'College(s)' in self.info_dict.keys():
            self.entity_dict['EDUCATION'] += [{'dict_key':'College(s)', 'type':'ORG', 'entity_list':[]}]
            
    def _get_sub_entity_dict(self, entity, dict_key):
        key_association_dict = {'NAME':{'Birth name', 'Born', 'Born:','Name','Name(s)','Full name'},
                                'BIRTH_PLACE':{'Born', 'Born:' ,'Home town'},
                                'BIRTH_DATE':{'Born', 'Born:'},
                                'CHILDREN':{'Children'},
                                'SPOUSES':{'Spouse','Spouse(s)','Spouses'},
                                'PARENTS':{'Parent','Parent(s)','Parents','Father','Father’s name','Mother','Mother’s name'},
                                'EDUCATION':{'Education','High school','High school:','Law School','School','Schools',
                                             'College','College(s)','Colleges','Alma mater','Almat mater','Alma\xa0mater'}}
        
    def _filter_NAME(self, text):
        '''cannot simply filter noise due to many possible candidates'''
        return True
    def _filter_BIRTH_DATE(self, text):
        return True
    def _filter_BIRTH_PLACE(self, text):
        return True
    def _filter_CHILDREN(self, text):
#         return False # let's assume we don't find any noisy text under info boxes children till we do an analysis
        return not self._is_name(text)
    def _filter_SPOUSES(self, text):
        return not self._is_name(text)
    def _filter_PARENTS(self, text):
        return not self._is_name(text)
    def _filter_EDUCATION(self, text):
        return True
    def _is_name(self, text):
        p = regex.compile(r"\p{Lu}") # To support (currently) 1702 uppercase letters
#         p = regex.compile(r"[[:upper:]]") # To support (currently) 1822 uppercase letters
        if p.match(text):
            return True
        else:
            return False

In [0]:
#experiment_code
# pg = PageContents('https://en.wikipedia.org/wiki/James_R._Alexander')
# pg = PageContents('https://en.wikipedia.org/wiki/Brad_Pitt')
# pg = PageContents('https://en.wikipedia.org/wiki/Mohamed_Abdel-El')
# pg = PageContents('https://en.wikipedia.org/wiki/Donald_Trump')
# info_card = InfoCard(pg)
# entities = PrivateEntities(info_card).entity_dict
# print(info_card.info_table['Born'], entities)

# pg = PageContents('https://en.wikipedia.org/wiki/Brad_Pitt')
# info_card = InfoCard(pg)
# entities = PrivateEntities(info_card).entity_dict
# print(info_card.info_table['Born'], entities)

pg = PageContents('http://en.wikipedia.org/wiki/Gajendra_Singh_Rajukhedi')
info_card = InfoCard(pg)
entities = PrivateEntities(info_card).entity_dict
print(info_card.info_table)
print(entities)

info card is scraped successfully
{'Preceded by': ['Chhatar Singh Darbar'], 'Succeeded by': ['Savitri Thakur'], 'Constituency': ['Dhar'], 'Born': ['', '1964-12-11', ')', '11 December 1964', '', 'Rajukhedi, Dhar'], 'Political party': ['Indian National Congress'], 'Spouse(s)': ['Gayatri Singh', 'm.', '1990)'], 'Children': ['2 daughters and 2 sons'], 'Mother': ['Ramkunwar Devi'], 'Father': ['Onkar Singh'], 'Residence': [], 'Education': ['B. A.', '', 'LLB'], 'Alma mater': ['Devi Ahilya University'], 'Profession': ['Politician', ',', 'Advocate']}
{'NAME': [{'dict_key': 'Born', 'type': 'PERSON', 'entity_list': ['Rajukhedi']}], 'BIRTH_DATE': [{'dict_key': 'Born', 'type': 'DATE', 'entity_list': ['1964-12-11']}], 'BIRTH_PLACE': [{'dict_key': 'Born', 'type': 'GPE', 'entity_list': ['Rajukhedi, Dhar']}], 'CHILDREN': [{'dict_key': 'Children', 'type': 'PERSON', 'entity_list': []}], 'SPOUSES': [{'dict_key': 'Spouse(s)', 'type': 'PERSON', 'entity_list': ['Gayatri Singh']}], 'PARENTS': [{'dict_key': 'F

In [0]:
#experiment_code
if not os.path.exists('entity_presence_score.csv'):
    wget.download('https://github.com/ratmcu/wiki_ner/blob/master/entity_presence_score.csv?raw=true')
entity_presence_score_df = pd.read_csv('entity_presence_score.csv')
entity_presence_score_df.iloc[:,1:].groupby('SCORE').count()
indices = entity_presence_score_df.iloc[:,1:].groupby('SCORE').groups[7.0]
for index in indices:
    url = 'http://en.wikipedia.org'+entity_presence_score_df.iloc[index]['LINK']
    try:
        urllib.request.urlopen(url, timeout = 5)
    except:
        raise Exception('page not found!!!!!!!!!')
    pg = PageContents(url)
    
    if not pg.table:
        raise Exception('unable to find a info table in the page')
# pg = PageContents('https://en.wikipedia.org/wiki/Donald_Trump')
    else:
        print(url)
        info_card = InfoCard(pg)
        print(info_card.info_table)
        print(PrivateEntities(info_card).entity_dict)

http://en.wikipedia.org/wiki/Carl_Roberts_(diplomat)
info card is scraped successfully
{'Preceded by': ['Ronald Michael Sanders'], 'Succeeded by': ['Althea Allison Vanderpoole Banahene'], 'Born': ['', '1948-10-13', ')', 'October 13, 1948', '', "St. John's, Antigua and Barbuda"], 'Spouse(s)': ['m 1974, Pauline Margaret A.'], 'Children': ['two sons one daughter'], 'Mother': ['Audrey E. Roberts'], 'Father': ['Arthur E. S. Roberts'], 'Education': ['Antigua Grammar School'], 'Alma mater': ['', '1982:', 'Bachelor of Engineering', 'Hons magna cum laude of the', 'Northeastern University', 'Boston', '', '1995', 'Master of Business Administration', 'of the', 'University of the West Indies', ',', 'Barbados', '.', '', '2004:', 'Doctor of Business Administration', 'of the', 'University of Bradford', '.', '']}
{'NAME': [{'dict_key': 'Born', 'type': 'PERSON', 'entity_list': []}], 'BIRTH_DATE': [{'dict_key': 'Born', 'type': 'DATE', 'entity_list': ['1948-10-13', 'October 13, 1948']}], 'BIRTH_PLACE': [{